Scraping from http://fbref.com

Data:
- outfield players stats
- goalkeepers stats
- team stats (of and against)

Original code by parth1902:
https://github.com/parth1902/Scrape-FBref-data

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [2]:
#standard(stats)
stats = ["player","nationality","position","team","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]
stats3 = ["players_used","possession","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"] 
#goalkeeping(keepers)
keepers = ["player","nationality","position","team","age","birth_year","gk_games","gk_games_starts","gk_minutes","gk_goals_against","gk_goals_against_per90","gk_shots_on_target_against","gk_saves","gk_save_pct","gk_wins","gk_ties","gk_losses","gk_clean_sheets","gk_clean_sheets_pct","gk_pens_att","gk_pens_allowed","gk_pens_saved","gk_pens_missed"]
keepers3 = ["players_used","gk_games","gk_games_starts","gk_minutes","gk_goals_against","gk_goals_against_per90","gk_shots_on_target_against","gk_saves","gk_save_pct","gk_wins","gk_ties","gk_losses","gk_clean_sheets","gk_clean_sheets_pct","gk_pens_att","gk_pens_allowed","gk_pens_saved","gk_pens_missed"]
#advance goalkeeping(keepersadv)
keepersadv = ["player","nationality","position","team","age","birth_year","minutes_90s","gk_goals_against","gk_pens_allowed","gk_free_kick_goals_against","gk_corner_kick_goals_against","gk_own_goals_against","gk_psxg","gk_psnpxg_per_shot_on_target_against","gk_psxg_net","gk_psxg_net_per90","gk_passes_completed_launched","gk_passes_launched","gk_passes_pct_launched","gk_passes","gk_passes_throws","gk_pct_passes_launched","gk_passes_length_avg","gk_goal_kicks","gk_pct_goal_kicks_launched","gk_goal_kick_length_avg","gk_crosses","gk_crosses_stopped","gk_crosses_stopped_pct","gk_def_actions_outside_pen_area","gk_def_actions_outside_pen_area_per90","gk_avg_distance_def_actions"]
keepersadv2 = ["minutes_90s","gk_goals_against","gk_pens_allowed","gk_free_kick_goals_against","gk_corner_kick_goals_against","gk_own_goals_against","gk_psxg","gk_psnpxg_per_shot_on_target_against","gk_psxg_net","gk_psxg_net_per90","gk_passes_completed_launched","gk_passes_launched","gk_passes_pct_launched","gk_passes","gk_passes_throws","gk_pct_passes_launched","gk_passes_length_avg","gk_goal_kicks","gk_pct_goal_kicks_launched","gk_goal_kick_length_avg","gk_crosses","gk_crosses_stopped","gk_crosses_stopped_pct","gk_def_actions_outside_pen_area","gk_def_actions_outside_pen_area_per90","gk_avg_distance_def_actions"]
#shooting(shooting)
shooting = ["player","nationality","position","team","age","birth_year","minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting2 = ["minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting3 = ["goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
#passing(passing)
passing = ["player","nationality","position","team","age","birth_year","minutes_90s","passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
passing2 = ["passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
#passtypes(passing_types)
passing_types = ["player","nationality","position","team","age","birth_year","minutes_90s","passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
passing_types2 = ["passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
#goal and shot creation(gca)
gca = ["player","nationality","position","team","age","birth_year","minutes_90s","sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
gca2 = ["sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
#defensive actions(defense)
defense = ["player","nationality","position","team","age","birth_year","minutes_90s","tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
defense2 = ["tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
#possession(possession)
possession = ["player","nationality","position","team","age","birth_year","minutes_90s","touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
possession2 = ["touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
#playingtime(playingtime)
playingtime = ["player","nationality","position","team","age","birth_year","minutes_90s","games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
playingtime2 = ["games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
#miscallaneous(misc)
misc = ["player","nationality","position","team","age","birth_year","minutes_90s","cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]
misc2 = ["cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]

def get_tables(url,text):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    res = requests.get(url, headers = headers)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    
    team_table = all_tables[0]
    team_vs_table = all_tables[1]
    player_table = all_tables[2]
    if text == 'for':
      return player_table, team_table
    if text == 'vs':
      return player_table, team_vs_table

def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):
    
            for f in features_wanted_player:
                cell = row.find("td",{"data-stat": f})
                if(cell == None):
                  continue
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')&(f!='team')):
                    text = float(text.replace(',',''))                  
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player

def get_frame_team(features, team_table):
    pre_df_squad = dict()
    #Note: features does not contain squad name, it requires special treatment
    features_wanted_squad = features
    rows_squad = team_table.find_all('tr')
    for row in rows_squad:
        if(row.find('th',{"scope":"row"}) != None):
            name = row.find('th',{"data-stat":"team"}).text.strip().encode().decode("utf-8")
            if 'team' in pre_df_squad:
                pre_df_squad['team'].append(name)
            else:
                pre_df_squad['team'] = [name]
            for f in features_wanted_squad:
                cell = row.find("td",{"data-stat": f})
                if(cell == None):
                  continue
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')&(f!='team')):
                    text = float(text.replace(',',''))
                if f in pre_df_squad:
                    pre_df_squad[f].append(text)
                else:
                    pre_df_squad[f] = [text]
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    return df_squad

def get_frame_team_2(features, team_table):
    pre_df_squad = dict()
    #Note: features does not contain squad name, it requires special treatment
    features_wanted_squad = features
    rows_squad = team_table.find_all('tr')
    for row in rows_squad:
        if(row.find('th',{"scope":"row"}) != None):
            name = row.find('th',{"data-stat":"squad"}).text.strip().encode().decode("utf-8")
            if 'squad' in pre_df_squad:
                pre_df_squad['squad'].append(name)
            else:
                pre_df_squad['squad'] = [name]
            for f in features_wanted_squad:
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')&(f!='team')):
                    text = float(text.replace(',',''))
                    print(f)
                if f in pre_df_squad:
                    pre_df_squad[f].append(text)
                else:
                    pre_df_squad[f] = [text]
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    return df_squad

def frame_for_category(category,top,end,features):
    url = (top + category + end)
    player_table, team_table = get_tables(url,'for')
    df_player = get_frame(features, player_table)
    return df_player

def frame_for_category_team(category,top,end,features,text):
    url = (top + category + end)
    player_table, team_table = get_tables(url,text)
    df_team = get_frame_team(features, team_table)
    return df_team




#Function to get the player data for outfield player, includes all categories - standard stats, shooting
#passing, passing types, goal and shot creation, defensive actions, possession, and miscallaneous
def get_outfield_data(top, end):
    df1 = frame_for_category('stats',top,end,stats)
    df2 = frame_for_category('shooting',top,end,shooting2)
    df3 = frame_for_category('passing',top,end,passing2)
    df4 = frame_for_category('passing_types',top,end,passing_types2)
    df5 = frame_for_category('gca',top,end,gca2)
    df6 = frame_for_category('defense',top,end,defense2)
    df7 = frame_for_category('possession',top,end,possession2)
    df8 = frame_for_category('misc',top,end,misc2)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df


#Function to get keeping and advance goalkeeping data
def get_keeper_data(top,end):
    df1 = frame_for_category('keepers',top,end,keepers)
    df2 = frame_for_category('keepersadv',top,end,keepersadv2)
    df = pd.concat([df1, df2], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df



#Function to get team-wise data accross all categories as mentioned above
def get_team_data(top,end,text):
    df1 = frame_for_category_team('stats',top,end,stats3,text)
    df2 = frame_for_category_team('keepers',top,end,keepers3,text)
    df3 = frame_for_category_team('keepersadv',top,end,keepersadv2,text)
    df4 = frame_for_category_team('shooting',top,end,shooting3,text)
    df5 = frame_for_category_team('passing',top,end,passing2,text)
    df6 = frame_for_category_team('passing_types',top,end,passing_types2,text)
    df7 = frame_for_category_team('gca',top,end,gca2,text)
    df8 = frame_for_category_team('defense',top,end,defense2,text)
    df9 = frame_for_category_team('possession',top,end,possession2,text)
    df10 = frame_for_category_team('misc',top,end,misc2,text)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df

In [3]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')

df_outfield.to_csv('fbref_data/outfield_players.csv', index=False)

df_outfield

,player,nationality,position,team,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,James Abankwah,ie IRL,DF,Udinese,19-099,2004,1.0,0.0,5.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Oliver Abildgaard,dk DEN,MF,Hellas Verona,26-319,1996,6.0,2.0,197.0,0.0,...,3.0,1.0,0.0,0.0,0.0,0.0,15.0,15.0,5.0,75.0
2,Tammy Abraham,eng ENG,FW,Roma,25-205,1997,31.0,22.0,1905.0,7.0,...,30.0,44.0,10.0,1.0,0.0,0.0,41.0,66.0,57.0,53.7
3,Christian Acella,it ITA,MF,Cremonese,20-292,2002,1.0,0.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Francesco Acerbi,it ITA,DF,Inter,35-074,1988,24.0,20.0,1915.0,0.0,...,12.0,10.0,1.0,0.0,0.0,0.0,122.0,69.0,32.0,68.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,Petar Zovko,ba BIH,GK,Spezia,21-031,2002,1.0,0.0,74.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
576,Szymon Żurkowski,pl POL,MF,Fiorentina,25-212,1997,2.0,0.0,32.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,50.0
577,Szymon Żurkowski,pl POL,MF,Spezia,25-212,1997,6.0,2.0,240.0,0.0,...,8.0,5.0,0.0,0.0,0.0,0.0,18.0,3.0,2.0,60.0
578,Milan Đurić,ba BIH,FW,Hellas Verona,32-338,1990,21.0,8.0,863.0,1.0,...,21.0,20.0,4.0,0.0,0.0,0.0,19.0,132.0,35.0,79.0


In [4]:
df_keeper = get_keeper_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')

df_keeper.to_csv('fbref_data/keepers_players.csv', index=False)

df_keeper

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Emil Audero,it ITA,GK,Sampdoria,26-097,1997,25.0,25.0,2250.0,39.0,...,39.1,200.0,57.0,44.2,356.0,24.0,6.7,23.0,0.92,13.9
1,Francesco Bardi,it ITA,GK,Bologna,31-097,1992,1.0,1.0,90.0,0.0,...,35.6,9.0,44.4,33.4,10.0,0.0,0.0,0.0,0.00,0.0
2,Marco Carnesecchi,it ITA,GK,Cremonese,22-298,2000,22.0,22.0,1980.0,38.0,...,40.9,159.0,61.6,50.7,305.0,23.0,7.5,21.0,0.95,14.0
3,Andrea Consigli,it ITA,GK,Sassuolo,36-088,1987,29.0,29.0,2610.0,43.0,...,34.2,249.0,32.5,33.4,409.0,24.0,5.9,26.0,0.90,14.1
4,Alessio Cragno,it ITA,GK,Monza,28-301,1994,1.0,1.0,90.0,3.0,...,22.6,6.0,16.7,33.3,7.0,0.0,0.0,2.0,2.00,25.7
5,Michele Di Gregorio,it ITA,GK,Monza,25-272,1997,30.0,30.0,2700.0,40.0,...,32.6,196.0,35.7,33.7,430.0,13.0,3.0,23.0,0.77,12.7
6,Bartłomiej Drągowski,pl POL,GK,Spezia,25-249,1997,28.0,28.0,2406.0,43.0,...,34.5,169.0,62.7,48.0,401.0,11.0,2.7,29.0,1.08,14.2
7,Wladimiro Falcone,it ITA,GK,Lecce,28-013,1995,31.0,31.0,2790.0,38.0,...,41.9,225.0,77.3,52.4,441.0,22.0,5.0,35.0,1.13,13.7
8,Pierluigi Gollini,it ITA,GK,Fiorentina,28-038,1995,3.0,3.0,270.0,2.0,...,32.4,11.0,63.6,49.3,24.0,1.0,4.2,0.0,0.00,9.3
9,Pierluigi Gollini,it ITA,GK,Napoli,28-038,1995,1.0,1.0,90.0,0.0,...,31.7,5.0,0.0,15.4,12.0,1.0,8.3,0.0,0.00,2.0


In [5]:
df_team = get_team_data('https://fbref.com/en/comps/11/','/Serie-A-Stats', 'for')

df_team.to_csv('fbref_data/teams.csv', index=False)

df_team

,team,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Atalanta,25.0,50.0,31.0,341.0,2790.0,50.0,32.0,6.0,8.0,...,381.0,320.0,44.0,6.0,3.0,2.0,1840.0,472.0,389.0,54.8
1,Bologna,27.0,53.4,31.0,341.0,2790.0,39.0,32.0,4.0,4.0,...,384.0,355.0,62.0,3.0,8.0,2.0,1732.0,306.0,379.0,44.7
2,Cremonese,32.0,43.2,31.0,341.0,2790.0,27.0,13.0,4.0,6.0,...,393.0,324.0,58.0,4.0,5.0,1.0,1708.0,462.0,594.0,43.8
3,Empoli,31.0,47.4,31.0,341.0,2790.0,25.0,13.0,1.0,1.0,...,354.0,365.0,39.0,1.0,4.0,1.0,1471.0,330.0,412.0,44.5
4,Fiorentina,29.0,56.6,31.0,341.0,2790.0,35.0,25.0,4.0,6.0,...,392.0,437.0,40.0,3.0,4.0,2.0,1684.0,504.0,425.0,54.3
5,Hellas Verona,36.0,42.0,31.0,341.0,2790.0,24.0,18.0,1.0,1.0,...,456.0,318.0,62.0,1.0,1.0,1.0,1861.0,615.0,619.0,49.8
6,Inter,24.0,56.2,31.0,341.0,2790.0,50.0,35.0,4.0,5.0,...,366.0,343.0,58.0,4.0,3.0,2.0,1609.0,443.0,344.0,56.3
7,Juventus,29.0,48.6,31.0,341.0,2790.0,47.0,37.0,3.0,5.0,...,363.0,334.0,52.0,5.0,2.0,0.0,1594.0,390.0,381.0,50.6
8,Lazio,22.0,51.9,31.0,341.0,2790.0,48.0,29.0,5.0,7.0,...,320.0,395.0,42.0,5.0,1.0,0.0,1631.0,319.0,319.0,50.0
9,Lecce,29.0,41.7,31.0,341.0,2790.0,24.0,17.0,1.0,2.0,...,452.0,362.0,54.0,2.0,5.0,3.0,1742.0,466.0,582.0,44.5


In [6]:
df_vsteam = get_team_data('https://fbref.com/en/comps/11/','/Serie-A-Stats', 'vs')

df_vsteam.to_csv('fbref_data/teams_vs.csv', index=False)

df_vsteam

,team,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,vs Atalanta,25.0,50.0,31.0,341.0,2790.0,34.0,26.0,3.0,3.0,...,338.0,356.0,40.0,1.0,8.0,1.0,1865.0,389.0,472.0,45.2
1,vs Bologna,27.0,46.6,31.0,341.0,2790.0,37.0,22.0,7.0,8.0,...,376.0,365.0,52.0,5.0,4.0,1.0,1687.0,379.0,306.0,55.3
2,vs Cremonese,32.0,56.8,31.0,341.0,2790.0,56.0,38.0,5.0,5.0,...,345.0,372.0,52.0,4.0,6.0,0.0,1745.0,594.0,462.0,56.3
3,vs Empoli,31.0,52.6,31.0,341.0,2790.0,39.0,27.0,2.0,4.0,...,384.0,341.0,60.0,2.0,1.0,0.0,1607.0,412.0,330.0,55.5
4,vs Fiorentina,29.0,43.4,31.0,341.0,2790.0,34.0,24.0,4.0,4.0,...,460.0,371.0,78.0,2.0,6.0,2.0,1625.0,425.0,504.0,45.7
5,vs Hellas Verona,36.0,58.0,31.0,341.0,2790.0,43.0,39.0,0.0,1.0,...,337.0,438.0,32.0,1.0,1.0,2.0,1728.0,619.0,615.0,50.2
6,vs Inter,24.0,43.8,31.0,341.0,2790.0,32.0,27.0,3.0,3.0,...,368.0,336.0,29.0,3.0,5.0,1.0,1443.0,344.0,443.0,43.7
7,vs Juventus,29.0,51.4,31.0,341.0,2790.0,26.0,22.0,1.0,2.0,...,348.0,328.0,39.0,0.0,5.0,0.0,1556.0,381.0,390.0,49.4
8,vs Lazio,22.0,48.1,31.0,341.0,2790.0,21.0,14.0,1.0,1.0,...,421.0,304.0,55.0,1.0,7.0,1.0,1677.0,319.0,319.0,50.0
9,vs Lecce,29.0,58.3,31.0,341.0,2790.0,35.0,25.0,5.0,5.0,...,378.0,425.0,57.0,4.0,2.0,2.0,1694.0,582.0,466.0,55.5


In [7]:
print('Outfield player columns')
print(df_outfield.columns.values)

print('Keeper player columns')
print(df_keeper.columns.values)

print('Team columns')
print(df_team.columns.values)

print('Vs Team columns')
print(df_team.columns.values)

Outfield player columns
['player' 'nationality' 'position' 'team' 'age' 'birth_year' 'games'
 'games_starts' 'minutes' 'goals' 'assists' 'pens_made' 'pens_att'
 'cards_yellow' 'cards_red' 'goals_per90' 'assists_per90'
 'goals_assists_per90' 'goals_pens_per90' 'goals_assists_pens_per90' 'xg'
 'npxg' 'xg_per90' 'npxg_per90' 'minutes_90s' 'shots_on_target'
 'shots_free_kicks' 'shots_on_target_pct' 'shots_on_target_per90'
 'goals_per_shot' 'goals_per_shot_on_target' 'npxg_per_shot' 'xg_net'
 'npxg_net' 'passes_completed' 'passes' 'passes_pct'
 'passes_total_distance' 'passes_progressive_distance'
 'passes_completed_short' 'passes_short' 'passes_pct_short'
 'passes_completed_medium' 'passes_medium' 'passes_pct_medium'
 'passes_completed_long' 'passes_long' 'passes_pct_long' 'assisted_shots'
 'passes_into_final_third' 'passes_into_penalty_area'
 'crosses_into_penalty_area' 'progressive_passes' 'passes_live'
 'passes_dead' 'passes_free_kicks' 'through_balls' 'passes_switches'
 'crosses' 'corn

Scrape data for past seasons. 

Do not repeat if data is already present.

In [ ]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/11/2021-2022/','/2021-2022-Serie-A-Stats')

df_outfield.to_csv('fbref_data/season2122/outfield_players.csv', index=False)

df_outfield

In [9]:
df_keeper = get_keeper_data('https://fbref.com/en/comps/11/2021-2022/','/2021-2022-Serie-A-Stats')

df_keeper.to_csv('fbref_data/season2122/keepers_players.csv', index=False)

df_keeper

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Emil Audero,it ITA,GK,Sampdoria,24,1997,29.0,29.0,2560.0,48.0,...,36.3,212.0,67.9,47.6,418.0,22.0,5.3,25.0,0.88,13.5
1,Nicola Bagnolini,it ITA,GK,Bologna,17,2004,1.0,0.0,3.0,0.0,...,61.0,1.0,100.0,60.0,3.0,0.0,0.0,0.0,0.00,0.0
2,Francesco Bardi,it ITA,GK,Bologna,29,1992,2.0,2.0,177.0,2.0,...,33.0,13.0,38.5,32.1,24.0,2.0,8.3,0.0,0.00,5.5
3,Vid Belec,si SVN,GK,Salernitana,31,1990,23.0,21.0,1938.0,49.0,...,39.5,182.0,73.1,49.9,329.0,12.0,3.6,14.0,0.65,12.7
4,Alessandro Berardi,it ITA,GK,Hellas Verona,30,1991,1.0,1.0,90.0,3.0,...,39.7,7.0,100.0,58.7,12.0,0.0,0.0,0.0,0.00,0.0
5,Etrit Berisha,al ALB,GK,Torino,32,1989,10.0,10.0,900.0,8.0,...,39.4,78.0,87.2,58.6,148.0,7.0,4.7,10.0,1.00,14.6
6,Andrea Consigli,it ITA,GK,Sassuolo,34,1987,37.0,37.0,3322.0,63.0,...,30.9,246.0,31.7,28.8,491.0,21.0,4.3,30.0,0.81,13.4
7,Alessio Cragno,it ITA,GK,Cagliari,27,1994,35.0,35.0,3150.0,63.0,...,39.4,316.0,73.4,48.2,484.0,22.0,4.5,33.0,0.94,15.0
8,Bartłomiej Drągowski,pl POL,GK,Fiorentina,23,1997,7.0,7.0,556.0,8.0,...,30.8,39.0,43.6,40.2,61.0,2.0,3.3,20.0,3.24,22.3
9,Wladimiro Falcone,it ITA,GK,Sampdoria,26,1995,10.0,9.0,855.0,14.0,...,33.2,74.0,70.3,45.0,164.0,7.0,4.3,2.0,0.21,10.2


In [10]:
df_team = get_team_data('https://fbref.com/en/comps/11/2021-2022/','/2021-2022-Serie-A-Stats', 'for')

df_team.to_csv('fbref_data/season2122/teams.csv', index=False)

df_team

,team,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Atalanta,32.0,55.0,38.0,418.0,3420.0,62.0,48.0,5.0,6.0,...,513.0,454.0,68.0,3.0,6.0,3.0,2292.0,685.0,518.0,56.9
1,Bologna,36.0,50.6,38.0,418.0,3420.0,43.0,34.0,4.0,5.0,...,443.0,497.0,56.0,3.0,9.0,2.0,1972.0,492.0,520.0,48.6
2,Cagliari,33.0,44.5,38.0,418.0,3420.0,34.0,26.0,3.0,4.0,...,549.0,490.0,70.0,3.0,10.0,1.0,1995.0,696.0,742.0,48.4
3,Empoli,28.0,47.4,38.0,418.0,3420.0,47.0,27.0,7.0,7.0,...,506.0,463.0,83.0,6.0,10.0,6.0,1998.0,427.0,549.0,43.8
4,Fiorentina,28.0,57.7,38.0,418.0,3420.0,59.0,33.0,9.0,12.0,...,464.0,565.0,57.0,8.0,5.0,2.0,1885.0,469.0,456.0,50.7
5,Genoa,40.0,43.9,38.0,418.0,3420.0,26.0,19.0,6.0,7.0,...,566.0,499.0,72.0,4.0,5.0,2.0,2228.0,635.0,743.0,46.1
6,Hellas Verona,31.0,50.6,38.0,418.0,3420.0,63.0,44.0,7.0,8.0,...,560.0,429.0,97.0,6.0,4.0,2.0,2413.0,689.0,733.0,48.5
7,Inter,27.0,56.5,38.0,418.0,3420.0,83.0,57.0,7.0,11.0,...,466.0,412.0,57.0,7.0,4.0,1.0,2004.0,549.0,476.0,53.6
8,Juventus,32.0,51.5,38.0,418.0,3420.0,56.0,37.0,5.0,6.0,...,508.0,503.0,74.0,5.0,7.0,1.0,2008.0,550.0,440.0,55.6
9,Lazio,27.0,55.4,38.0,418.0,3420.0,74.0,48.0,7.0,9.0,...,441.0,450.0,44.0,6.0,6.0,1.0,1941.0,411.0,427.0,49.0


In [11]:
df_vsteam = get_team_data('https://fbref.com/en/comps/11/2021-2022/','/2021-2022-Serie-A-Stats', 'vs')

df_vsteam.to_csv('fbref_data/season2122/teams_vs.csv', index=False)

df_vsteam

,team,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,vs Atalanta,32.0,44.9,38.0,418.0,3420.0,45.0,29.0,5.0,6.0,...,491.0,488.0,69.0,4.0,6.0,3.0,2240.0,518.0,685.0,43.1
1,vs Bologna,36.0,49.3,38.0,418.0,3420.0,53.0,32.0,8.0,9.0,...,527.0,416.0,88.0,8.0,5.0,1.0,2032.0,520.0,492.0,51.4
2,vs Cagliari,33.0,55.6,38.0,418.0,3420.0,67.0,41.0,7.0,10.0,...,526.0,524.0,76.0,6.0,4.0,0.0,2112.0,742.0,696.0,51.6
3,vs Empoli,28.0,52.9,38.0,418.0,3420.0,64.0,44.0,5.0,10.0,...,488.0,482.0,66.0,7.0,7.0,3.0,2051.0,549.0,427.0,56.3
4,vs Fiorentina,28.0,41.8,38.0,418.0,3420.0,49.0,33.0,3.0,5.0,...,585.0,441.0,119.0,4.0,12.0,0.0,1825.0,456.0,469.0,49.3
5,vs Genoa,40.0,56.1,38.0,418.0,3420.0,58.0,35.0,4.0,5.0,...,523.0,530.0,59.0,4.0,7.0,1.0,2257.0,743.0,635.0,53.9
6,vs Hellas Verona,31.0,49.4,38.0,418.0,3420.0,57.0,38.0,3.0,4.0,...,463.0,534.0,32.0,3.0,8.0,2.0,2206.0,733.0,689.0,51.5
7,vs Inter,27.0,43.4,38.0,418.0,3420.0,31.0,19.0,4.0,4.0,...,440.0,441.0,34.0,3.0,11.0,1.0,1767.0,476.0,549.0,46.4
8,vs Juventus,32.0,48.3,38.0,418.0,3420.0,36.0,21.0,4.0,7.0,...,525.0,471.0,35.0,5.0,6.0,1.0,1894.0,440.0,550.0,44.4
9,vs Lazio,27.0,44.4,38.0,418.0,3420.0,57.0,40.0,5.0,6.0,...,474.0,418.0,73.0,6.0,9.0,3.0,1956.0,427.0,411.0,51.0


In [12]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/11/2020-2021/','/2020-2021-Serie-A-Stats')
df_outfield.to_csv('fbref_data/season2021/outfield_players.csv', index=False)

df_keeper = get_keeper_data('https://fbref.com/en/comps/11/2020-2021/','/2020-2021-Serie-A-Stats')
df_keeper.to_csv('fbref_data/season2021/keepers_players.csv', index=False)

df_team = get_team_data('https://fbref.com/en/comps/11/2020-2021/','/2020-2021-Serie-A-Stats', 'for')
df_team.to_csv('fbref_data/season2021/teams.csv', index=False)

df_vsteam = get_team_data('https://fbref.com/en/comps/11/2020-2021/','/2020-2021-Serie-A-Stats', 'vs')
df_vsteam.to_csv('fbref_data/season2021/teams_vs.csv', index=False)
